# Рекомендация тарифов

Цель - построить модель для задачи классификации, которая выберет подходящий тариф и  довести долю правильных ответов до 0.75.


### Описание данных

    сalls — количество звонков,
    minutes — суммарная длительность звонков в минутах,
    messages — количество sms-сообщений,
    mb_used — израсходованный интернет-трафик в Мб,
    is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

## Откроем и изучим файл

In [ ]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
from sklearn.dummy import DummyClassifier
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

In [ ]:
df = pd.read_csv('/datasets/users_behavior.csv')

In [ ]:
df

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
...,...,...,...,...,...
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [ ]:
df.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


##### Датасет имеет 5 столбцов и 3214 сторк. Пропусков нет.

## Разобьем данные на выборки

In [ ]:
features_train = df.drop('is_ultra', axis=1)
target_train = df['is_ultra']

In [ ]:
features_train, features_valid, target_train, target_valid = train_test_split(features_train, target_train,
                                                                             stratify=target_train, test_size=0.4,
                                                                             random_state=12345)
features_valid, features_test, target_valid, target_test = train_test_split(features_valid, target_valid,
                                                                           stratify=target_valid, test_size=0.5,
                                                                           random_state=12345)
display(target_train.shape, target_valid.shape, target_test.shape)

(1928,)

(643,)

(643,)

Разбили датасет на 3 выборки: обучающую, валидационную и тестовую.

## Исследуем модели

In [ ]:
best_model = None
best_result = 0
for depth in range(1, 11):
    model_decision = DecisionTreeClassifier(random_state=12345, max_depth=depth) 
    model_decision.fit(features_train, target_train) 
    predictions = model_decision.predict(features_valid)
    result = accuracy_score(target_valid, predictions) 
    if result > best_result:
        best_model_decision = model_decision
        best_result = result
print("Accuracy лучшей модели дерева решений:", best_result)

Accuracy лучшей модели дерева решений: 0.80248833592535


In [ ]:
best_model = None
best_result = 0
best_est = 0
best_depth = 0
for est in range(10, 51, 10):
    for depth in range (1, 11):
        model_forest = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        model_forest.fit(features_train, target_train) 
        predictions_valid = model_forest.predict(features_valid) 
        result = accuracy_score(target_valid, predictions_valid)
        if result > best_result:
            best_model_forest = model_forest
            best_result = result
            best_est = est
            best_depth = depth

print("Accuracy наилучшей модели случайного леса на валидационной выборке:", best_result)

Accuracy наилучшей модели случайного леса на валидационной выборке: 0.8211508553654744


In [ ]:
model_logistic = LogisticRegression(random_state=12345)
model_logistic.fit(features_train, target_train) 
predictions = model_logistic.predict(features_valid) 
result = accuracy_score(target_valid, predictions) 

print("Accuracy модели логистической регрессии на валидационной выборке:", result)

Accuracy модели логистической регрессии на валидационной выборке: 0.71850699844479


In [ ]:
best_model_decision.fit(pd.concat([features_train, features_valid]), pd.concat([target_train, target_valid]))

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=8,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=12345, splitter='best')

In [ ]:
best_model_forest.fit(pd.concat([features_train, features_valid]), pd.concat([target_train, target_valid]))

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=9, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=40,
                       n_jobs=None, oob_score=False, random_state=12345,
                       verbose=0, warm_start=False)

In [ ]:
model_logistic.fit(pd.concat([features_train, features_valid]), pd.concat([target_train, target_valid]))

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=12345, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

#### Вывод:

Обучая три разныемодели были полученны лучшие метрики качества:

- Качество модели "Дерево предсказания": 0.80
- Качество модели "Случайный лес": 0.82
- Качество модели "Логистическая регрессия": 0.71

Лучшей моделью получилась модель "Случайный лес"

## Проверим модель на тестовой выборке

In [ ]:
predictions_model_decision = best_model_decision.predict(features_test)
result = accuracy_score(target_test, predictions_model_decision)
print("Accuracy модели дерева решений на тестовой выборке:", result)

Accuracy модели дерева решений на тестовой выборке: 0.807153965785381


In [ ]:
predictions_model_forest = best_model_forest.predict(features_test)
result = accuracy_score(target_test, predictions_model_forest)
print("Accuracy модели случайного леса на тестовой выборке:", result)

Accuracy модели случайного леса на тестовой выборке: 0.8289269051321928


In [ ]:
predictions_model_logistic = model_logistic.predict(features_test)
result = accuracy_score(target_test, predictions_model_logistic)
print("Accuracy модели логистической регрессии на тестовой выборке:", result)

Accuracy модели логистической регрессии на тестовой выборке: 0.7060653188180405


#### Вывод
После проверки на тестовой выборке:

    Качество модели "Дерево предсказания": 0.80
    Качество модели "Случайный лес": 0.82
    Качество модели "Логистическая регрессия": 0.70
    
Лучшей моделью получилась модель "Случайный лес"

## Проверим модели на адекватность

In [ ]:
dummy_clf = DummyClassifier(strategy="most_frequent", random_state=0)
dummy_clf.fit(features_train, target_train)
dummy_clf.score(features_test, target_test)

0.6936236391912908

Простейшая модель ошибается в 30 %, выбранная в 17 %.

#### Общий вывод

    1. Разбили датасет на 3 выборки: обучающую - 60%, валидационную - 20% и тестовую - 20%.
    2. Исследовали модели дерева решений, случайного леса и логистической регрессии.
    3. Лучшее accuracy у случайного леса как на валидационной, так и на тестовой выборке.
